In [ ]:
#!/usr/bin/env python3
import os
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas

"""
===============================================================================
BACKTEST PERFORMANCE REPORT + TRADE / CASH / MARGIN / EQUITY IMPACT LEDGERS
ENHANCED WITH YEARLY DRAWDOWN ANALYSIS
===============================================================================

Inputs expected in BACKTEST_OUTPUT_ROOT:
- portfolio_equity.(csv|parquet)  [required]
- positions.(csv|parquet)         [required for margin/free cash]
- trades.csv                      [optional but recommended]

Also reads contract margins from:
- futures_contracts_full.parquet  [required for margin/free cash]

Outputs in REPORT_ROOT:
- performance_summary.csv          [includes overall max drawdown]
- monthly_returns.csv, yearly_returns.csv
- yearly_drawdowns.csv             [NEW: max drawdown per year]
- rolling_annualized_volatility.(csv|parquet)
- daily_account_ledger.(csv|parquet)
- trade_impact_ledger.csv
- charts (png), report.md, report.pdf

NEW FEATURES:
1. Maximum drawdown for entire strategy (already in performance_summary.csv)
2. Maximum drawdown per calendar year (new file: yearly_drawdowns.csv)
===============================================================================
"""

# ============================================================
# Configuration
# ============================================================

CONFIG = {
    "backtest_output_root": Path("./05-futures_backtest_position_sizing_mapped"),
    "report_root": Path("./06-reporting"),

    # Universe / contract specs for margin lookup
    "contracts_file": Path("./01-futures_universe/futures_contracts_full.parquet"),

    # Annualization
    "trading_days_per_year": 256,

    # Rolling windows for plots
    "rolling_vol_window": 32,

    # Charts / outputs
    "write_pdf": True,
    "write_markdown": True,
    "write_charts": True,

    # Margin column auto-detection candidates (first match wins)
    "margin_column_candidates": [
        "initial_margin",
        "init_margin",
        "margin",
        "initial_margin_usd",
        "margin_usd",
    ],
}

# ============================================================
# Data loading
# ============================================================

def _load_df_prefer_parquet(root: Path, stem: str) -> pd.DataFrame:
    pq = root / f"{stem}.parquet"
    csv = root / f"{stem}.csv"
    if pq.exists():
        return pd.read_parquet(pq)
    if csv.exists():
        return pd.read_csv(csv)
    raise FileNotFoundError(f"Missing {stem}.parquet or {stem}.csv in {root}")

def load_inputs(root: Path):
    portfolio = _load_df_prefer_parquet(root, "portfolio_equity")

    positions = None
    try:
        positions = _load_df_prefer_parquet(root, "positions")
    except Exception:
        positions = None

    trades = None
    trades_path = root / "trades.csv"
    if trades_path.exists():
        trades = pd.read_csv(trades_path)

    return portfolio, positions, trades

def normalize_portfolio_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [str(c).strip().lower() for c in df.columns]

    if "date" not in df.columns or "equity" not in df.columns:
        raise ValueError(f"portfolio_equity must include date and equity. Columns: {list(df.columns)}")

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"]).sort_values("date")

    df["equity"] = pd.to_numeric(df["equity"], errors="coerce")
    df = df.dropna(subset=["equity"])

    if "return" not in df.columns:
        df["return"] = df["equity"].pct_change()
    df["return"] = pd.to_numeric(df["return"], errors="coerce")

    # Optional columns (if present from your engine)
    for col in ["pnl", "commissions", "k_active", "tau_per_instrument"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    return df.reset_index(drop=True)

def normalize_positions_df(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty:
        return None
    df = df.copy()
    df.columns = [str(c).strip() for c in df.columns]
    if "date" not in [c.lower() for c in df.columns]:
        raise ValueError("positions must include a date column")
    # standardize date col name to 'date'
    date_col = next(c for c in df.columns if c.lower() == "date")
    if date_col != "date":
        df = df.rename(columns={date_col: "date"})
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"]).sort_values("date")
    # coerce positions to numeric ints (contracts)
    for c in df.columns:
        if c == "date":
            continue
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)
    return df.reset_index(drop=True)

def normalize_trades_df(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty:
        return None
    df = df.copy()
    df.columns = [str(c).strip().lower() for c in df.columns]
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
    for col in ["commission", "delta_contracts", "new_contracts", "prev_contracts"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

# ============================================================
# Margin lookup
# ============================================================

def load_margin_map(contracts_file: Path, candidates: list[str]) -> dict:
    if not contracts_file.exists():
        raise FileNotFoundError(f"contracts_file not found: {contracts_file}")

    df = pd.read_parquet(contracts_file)
    df.columns = [str(c).strip().lower() for c in df.columns]

    sym_col_candidates = ["symbol", "ticker", "root", "contract", "instrument"]
    sym_col = next((c for c in sym_col_candidates if c in df.columns), None)
    if sym_col is None:
        raise ValueError(f"Could not find symbol column in contracts file. Columns: {list(df.columns)}")

    margin_col = next((c for c in candidates if c in df.columns), None)
    if margin_col is None:
        # No margin in file; caller can still run, but margin_used will be NaN
        return {}

    out = {}
    for _, r in df.iterrows():
        sym = str(r[sym_col]).strip().upper()
        m = pd.to_numeric(r[margin_col], errors="coerce")
        if sym and pd.notna(m) and float(m) > 0:
            out[sym] = float(m)
    return out

# ============================================================
# Ledgers
# ============================================================

def compute_daily_margin_used(positions: pd.DataFrame, margin_map: dict) -> pd.DataFrame:
    """
    positions: columns: date, <TRADE_SYMBOL_1>, <TRADE_SYMBOL_2>, ...
    margin_used_t = sum_i abs(contracts_i,t) * margin_per_contract_i
    """
    if positions is None or positions.empty:
        return None

    pos = positions.copy()
    syms = [c for c in pos.columns if c != "date"]
    # margin vector aligned to columns (NaN if missing)
    m = np.array([margin_map.get(s.upper(), np.nan) for s in syms], dtype=float)

    # abs positions matrix
    mat = pos[syms].to_numpy(dtype=float)
    abs_mat = np.abs(mat)

    # margin used per day
    # if margin missing for any symbol, result becomes NaN; handle by treating missing margin as 0 if you prefer
    margin_used = np.nansum(abs_mat * m, axis=1)

    out = pd.DataFrame({
        "date": pos["date"],
        "margin_used": margin_used,
    })
    return out

def build_daily_account_ledger(portfolio: pd.DataFrame, positions: pd.DataFrame, margin_map: dict) -> pd.DataFrame:
    """
    Daily account statement:
    - equity (from portfolio)
    - pnl, commissions (from portfolio if present)
    - margin_used (from positions * margin_map)
    - free_cash = equity - margin_used
    - cash_before_trades = equity_prev + pnl_today
    - cash_after_trades = cash_before_trades - commissions_today
    """
    p = portfolio.copy()
    p = p.sort_values("date").reset_index(drop=True)

    # Margin
    if positions is not None and margin_map:
        m = compute_daily_margin_used(positions, margin_map)
        p = p.merge(m, on="date", how="left")
    else:
        p["margin_used"] = np.nan

    p["free_cash"] = p["equity"] - p["margin_used"]

    # Cash lens (futures-style): equity is your account value; trades consume commissions; margin is a requirement
    pnl_col = "pnl" if "pnl" in p.columns else None
    comm_col = "commissions" if "commissions" in p.columns else None

    p["equity_prev"] = p["equity"].shift(1)

    if pnl_col is not None:
        p["cash_before_trades"] = p["equity_prev"] + p[pnl_col]
    else:
        # fallback: approximate using equity change (less commissions unknown)
        p["cash_before_trades"] = p["equity_prev"]

    if comm_col is not None:
        p["cash_after_trades"] = p["cash_before_trades"] - p[comm_col]
    else:
        p["cash_after_trades"] = p["cash_before_trades"]

    # free cash before/after
    p["margin_used_prev"] = p["margin_used"].shift(1)
    p["free_cash_prev"] = p["equity_prev"] - p["margin_used_prev"]
    p["free_cash_after_trades"] = p["cash_after_trades"] - p["margin_used"]

    # tidy
    cols = [
        "date", "equity", "return",
        "pnl" if "pnl" in p.columns else None,
        "commissions" if "commissions" in p.columns else None,
        "margin_used", "free_cash",
        "cash_before_trades", "cash_after_trades",
        "free_cash_prev", "free_cash_after_trades",
        "k_active" if "k_active" in p.columns else None,
        "tau_per_instrument" if "tau_per_instrument" in p.columns else None,
    ]
    cols = [c for c in cols if c is not None and c in p.columns]
    return p[cols].copy()

def build_trade_impact_ledger(trades: pd.DataFrame, daily_ledger: pd.DataFrame, positions: pd.DataFrame, margin_map: dict) -> pd.DataFrame:
    """
    For each trade row, attach:
    - equity_before (prior day end), equity_after (same-day end)
    - cash_before_trades, cash_after_trades
    - margin_before (prior day), margin_after (same day)
    - free_cash_before, free_cash_after
    """
    if trades is None or trades.empty:
        return pd.DataFrame()

    t = trades.copy()
    # Try to use trade_symbol if present, else symbol
    sym_col = "trade_symbol" if "trade_symbol" in t.columns else ("symbol" if "symbol" in t.columns else None)
    if sym_col is None:
        sym_col = "trade_symbol"
        t[sym_col] = ""

    # Merge daily ledger (after trades)
    d = daily_ledger.copy()
    dcols = ["date", "equity", "margin_used", "free_cash", "cash_before_trades", "cash_after_trades", "free_cash_prev", "free_cash_after_trades"]
    dcols = [c for c in dcols if c in d.columns]
    t = t.merge(d[dcols], on="date", how="left")

    # Add before/after equity explicitly
    # equity_before = prior ledger equity (date-1)
    d_before = daily_ledger[["date", "equity", "margin_used", "free_cash"]].copy()
    d_before = d_before.rename(columns={
        "equity": "equity_before",
        "margin_used": "margin_before",
        "free_cash": "free_cash_before",
    })
    d_before["date"] = pd.to_datetime(d_before["date"])
    d_before["date_next"] = d_before["date"].shift(-1)  # not used
    # Better: merge by prior day using shift on ledger
    shifted = daily_ledger.copy()
    shifted["date"] = pd.to_datetime(shifted["date"])
    shifted = shifted.sort_values("date").reset_index(drop=True)
    shifted["equity_before"] = shifted["equity"].shift(1)
    shifted["margin_before"] = shifted["margin_used"].shift(1)
    shifted["free_cash_before"] = shifted["free_cash"].shift(1)
    shifted = shifted[["date", "equity_before", "margin_before", "free_cash_before"]]
    t = t.merge(shifted, on="date", how="left")

    # equity_after etc already in merged columns
    t = t.rename(columns={
        "equity": "equity_after",
        "margin_used": "margin_after",
        "free_cash": "free_cash_after",
    })

    # Commission column normalization
    if "commission" in t.columns:
        t["commission"] = pd.to_numeric(t["commission"], errors="coerce").fillna(0.0)

    # A simple "trade_day" flag
    t["is_trade_day"] = True

    # Sort for readability
    sort_cols = ["date"]
    if sym_col in t.columns:
        sort_cols.append(sym_col)
    return t.sort_values(sort_cols).reset_index(drop=True)

# ============================================================
# Core metrics & tables (existing)
# ============================================================

def compute_drawdown(equity: pd.Series) -> pd.Series:
    running_max = equity.cummax()
    return equity / running_max - 1.0

def compute_rolling_annualized_volatility(portfolio: pd.DataFrame, window_days: int = 42) -> pd.DataFrame:
    """
    Compute 2-month rolling annualized standard deviation of percentage returns.
    
    Parameters:
    -----------
    portfolio : DataFrame with 'date' and 'return' columns
    window_days : rolling window in trading days (default 42 ≈ 2 months)
    
    Returns:
    --------
    DataFrame with columns: date, rolling_ann_vol
    """
    df = portfolio[["date", "return"]].copy()
    df = df.sort_values("date").reset_index(drop=True)
    
    trading_days_per_year = CONFIG["trading_days_per_year"]
    
    # Rolling std of daily returns, annualized
    rolling_std = df["return"].rolling(window=window_days, min_periods=window_days).std()
    df["rolling_ann_vol"] = rolling_std * np.sqrt(trading_days_per_year)
    
    return df[["date", "rolling_ann_vol"]].copy()


def compute_performance_metrics(portfolio: pd.DataFrame, trading_days_per_year: int) -> dict:
    equity = portfolio["equity"].astype(float).reset_index(drop=True)
    rets = portfolio["return"].astype(float).reset_index(drop=True).dropna()
    n_days = int(rets.shape[0])

    start_eq = float(equity.iloc[0])
    end_eq = float(equity.iloc[-1])
    total_return = end_eq / start_eq - 1.0

    years = n_days / float(trading_days_per_year) if n_days > 0 else np.nan
    cagr = (end_eq / start_eq) ** (1.0 / years) - 1.0 if (years and years > 0) else np.nan

    mean_daily = float(rets.mean()) if n_days > 0 else np.nan
    std_daily = float(rets.std(ddof=1)) if n_days > 1 else np.nan

    ann_vol = std_daily * np.sqrt(trading_days_per_year) if np.isfinite(std_daily) else np.nan
    sharpe = (mean_daily / std_daily) * np.sqrt(trading_days_per_year) if (std_daily and std_daily > 0) else np.nan

    dd = compute_drawdown(equity)
    max_dd = float(dd.min()) if len(dd) else np.nan

    return {
        "start_date": portfolio["date"].iloc[0].date().isoformat(),
        "end_date": portfolio["date"].iloc[-1].date().isoformat(),
        "trading_days": n_days,
        "start_equity": start_eq,
        "end_equity": end_eq,
        "total_return": total_return,
        "cagr": cagr,
        "ann_vol": ann_vol,
        "sharpe": sharpe,
        "max_drawdown": max_dd,
    }

def monthly_and_yearly_returns(portfolio: pd.DataFrame):
    df = portfolio.copy().set_index("date")
    eq = df["equity"].astype(float)
    eq_m = eq.resample("M").last()
    ret_m = eq_m.pct_change()
    eq_y = eq.resample("Y").last()
    ret_y = eq_y.pct_change()

    monthly_tbl = ret_m.to_frame("ret")
    monthly_tbl["year"] = monthly_tbl.index.year
    monthly_tbl["month"] = monthly_tbl.index.month
    monthly_pivot = monthly_tbl.pivot(index="year", columns="month", values="ret").sort_index()

    yearly = ret_y.to_frame("yearly_return")
    yearly.index = yearly.index.year
    return monthly_pivot, yearly

def compute_yearly_drawdowns(portfolio: pd.DataFrame) -> pd.DataFrame:
    """
    Compute maximum drawdown for each calendar year.
    
    Parameters:
    -----------
    portfolio : DataFrame with 'date' and 'equity' columns
    
    Returns:
    --------
    DataFrame with columns: year, max_drawdown
    """
    df = portfolio.copy()
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    df['year'] = df['date'].dt.year
    
    yearly_dd = []
    
    for year in df['year'].unique():
        year_data = df[df['year'] == year].copy()
        if len(year_data) > 0:
            equity = year_data['equity'].astype(float)
            dd = compute_drawdown(equity)
            max_dd = float(dd.min()) if len(dd) > 0 else np.nan
            yearly_dd.append({
                'year': int(year),
                'max_drawdown': max_dd
            })
    
    return pd.DataFrame(yearly_dd)

def compute_yearly_risk_metrics(portfolio: pd.DataFrame, trading_days_per_year: int = 256) -> pd.DataFrame:
    """
    Compute annual risk-adjusted metrics for each calendar year:
    - Annual standard deviation (volatility)
    - Sharpe ratio
    - Calmar ratio (annual return / abs(max drawdown))
    
    Parameters:
    -----------
    portfolio : DataFrame with 'date', 'equity', and 'return' columns
    trading_days_per_year : int, default 256
    
    Returns:
    --------
    DataFrame with columns: year, annual_std_dev, sharpe_ratio, calmar_ratio
    """
    df = portfolio.copy()
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    df['year'] = df['date'].dt.year
    
    yearly_metrics = []
    
    for year in df['year'].unique():
        year_data = df[df['year'] == year].copy()
        
        if len(year_data) < 2:
            # Not enough data for this year
            yearly_metrics.append({
                'year': int(year),
                'annual_std_dev': np.nan,
                'sharpe_ratio': np.nan,
                'calmar_ratio': np.nan,
            })
            continue
        
        # Get returns and equity for the year
        returns = year_data['return'].dropna()
        equity = year_data['equity'].astype(float)
        
        if len(returns) < 2:
            yearly_metrics.append({
                'year': int(year),
                'annual_std_dev': np.nan,
                'sharpe_ratio': np.nan,
                'calmar_ratio': np.nan,
            })
            continue
        
        # Calculate daily statistics
        mean_daily = float(returns.mean())
        std_daily = float(returns.std(ddof=1))
        n_days = len(returns)
        
        # Annualize standard deviation
        annual_std_dev = std_daily * np.sqrt(trading_days_per_year) if np.isfinite(std_daily) else np.nan
        
        # Calculate Sharpe ratio (annualized)
        sharpe = (mean_daily / std_daily) * np.sqrt(trading_days_per_year) if (std_daily and std_daily > 0) else np.nan
        
        # Calculate annual return for this year
        start_equity = equity.iloc[0]
        end_equity = equity.iloc[-1]
        annual_return = (end_equity / start_equity) ** (trading_days_per_year / n_days) - 1.0 if n_days > 0 else np.nan
        
        # Calculate max drawdown for the year
        running_max = equity.cummax()
        drawdown = equity / running_max - 1.0
        max_dd = float(drawdown.min()) if len(drawdown) > 0 else np.nan
        
        # Calculate Calmar ratio (annual return / abs(max drawdown))
        calmar = annual_return / abs(max_dd) if (np.isfinite(max_dd) and max_dd != 0) else np.nan
        
        yearly_metrics.append({
            'year': int(year),
            'annual_std_dev': annual_std_dev,
            'sharpe_ratio': sharpe,
            'calmar_ratio': calmar,
        })
    
    return pd.DataFrame(yearly_metrics)

# ============================================================
# Charts (trade markers)
# ============================================================

def save_equity_curve_with_trade_markers(portfolio: pd.DataFrame, trades: pd.DataFrame, out_png: Path):
    df = portfolio.copy()
    plt.figure()
    plt.plot(df["date"], df["equity"])
    if trades is not None and not trades.empty and "date" in trades.columns:
        td = pd.to_datetime(trades["date"], errors="coerce").dropna().drop_duplicates().sort_values()
        for d in td:
            plt.axvline(d, linewidth=0.5)
    plt.title("Equity Curve (Trade Days Marked)")
    plt.xlabel("Date")
    plt.ylabel("Equity")
    plt.tight_layout()
    plt.savefig(out_png, dpi=150)
    plt.close()

def save_margin_and_free_cash(daily_ledger: pd.DataFrame, out_png: Path):
    plt.figure()
    plt.plot(daily_ledger["date"], daily_ledger["margin_used"])
    plt.title("Margin Used")
    plt.xlabel("Date")
    plt.ylabel("Margin Used")
    plt.tight_layout()
    plt.savefig(out_png, dpi=150)
    plt.close()

    # free cash
    out_png2 = out_png.with_name("free_cash.png")
    plt.figure()
    plt.plot(daily_ledger["date"], daily_ledger["free_cash"])
    plt.title("Free Cash (Equity - Margin)")
    plt.xlabel("Date")
    plt.ylabel("Free Cash")
    plt.tight_layout()
    plt.savefig(out_png2, dpi=150)
    plt.close()

# ============================================================
# Main
# ============================================================

def main():
    root = CONFIG["backtest_output_root"]
    report_root = CONFIG["report_root"]
    report_root.mkdir(parents=True, exist_ok=True)

    portfolio_df, positions_df, trades_df = load_inputs(root)
    portfolio_df = normalize_portfolio_df(portfolio_df)
    positions_df = normalize_positions_df(positions_df)
    trades_df = normalize_trades_df(trades_df)

    # Margin map
    margin_map = load_margin_map(CONFIG["contracts_file"], CONFIG["margin_column_candidates"])

    # Build ledgers
    daily_ledger = build_daily_account_ledger(portfolio_df, positions_df, margin_map)
    trade_impact = build_trade_impact_ledger(trades_df, daily_ledger, positions_df, margin_map)

    # Write ledgers
    daily_csv = report_root / "daily_account_ledger.csv"
    daily_pq = report_root / "daily_account_ledger.parquet"
    daily_ledger.to_csv(daily_csv, index=False)
    daily_ledger.to_parquet(daily_pq, index=False, compression="snappy")

    trade_csv = report_root / "trade_impact_ledger.csv"
    trade_impact.to_csv(trade_csv, index=False)
    
    # Compute and save rolling volatility
    rolling_vol_df = compute_rolling_annualized_volatility(portfolio_df, window_days=42)
    rolling_vol_csv = report_root / "rolling_annualized_volatility.csv"
    rolling_vol_pq = report_root / "rolling_annualized_volatility.parquet"
    rolling_vol_df.to_csv(rolling_vol_csv, index=False)
    rolling_vol_df.to_parquet(rolling_vol_pq, index=False, compression="snappy")


    # Existing summary outputs
    metrics = compute_performance_metrics(portfolio_df, CONFIG["trading_days_per_year"])
    monthly_pivot, yearly = monthly_and_yearly_returns(portfolio_df)
    
    # NEW: Compute yearly drawdowns
    yearly_drawdowns = compute_yearly_drawdowns(portfolio_df)
    
    # NEW: Compute yearly risk metrics
    yearly_risk_metrics = compute_yearly_risk_metrics(portfolio_df, CONFIG["trading_days_per_year"])

    summary_csv = report_root / "performance_summary.csv"
    pd.DataFrame([metrics]).to_csv(summary_csv, index=False)

    monthly_csv = report_root / "monthly_returns.csv"
    yearly_csv = report_root / "yearly_returns.csv"
    yearly_dd_csv = report_root / "yearly_drawdowns.csv"
    monthly_pivot.to_csv(monthly_csv, index=True)
    yearly.to_csv(yearly_csv, index=True)
    yearly_drawdowns.to_csv(yearly_dd_csv, index=False)
    # NEW: Save yearly risk metrics
    yearly_risk_csv = report_root / "yearly_risk_metrics.csv"
    yearly_risk_metrics.to_csv(yearly_risk_csv, index=False)

    # Charts
    if CONFIG["write_charts"]:
        save_equity_curve_with_trade_markers(portfolio_df, trades_df, report_root / "equity_curve_trade_markers.png")
        if "margin_used" in daily_ledger.columns:
            save_margin_and_free_cash(daily_ledger, report_root / "margin_used.png")

    print(f"Report outputs written to: {report_root.resolve()}")
    print(f"- {daily_csv.name}, {daily_pq.name}")
    print(f"- {trade_csv.name}")
    print(f"- {rolling_vol_csv.name}, {rolling_vol_pq.name}")
    print(f"- {summary_csv.name}, {monthly_csv.name}, {yearly_csv.name}, {yearly_dd_csv.name}, {yearly_risk_csv.name}")
    print(f"- {summary_csv.name}, {monthly_csv.name}, {yearly_csv.name}, {yearly_dd_csv.name}")

if __name__ == "__main__":
    main()

C:\Users\farty\AppData\Local\Temp\ipykernel_68140\1160959573.py:412: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  eq_m = eq.resample("M").last()
C:\Users\farty\AppData\Local\Temp\ipykernel_68140\1160959573.py:414: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  eq_y = eq.resample("Y").last()


Report outputs written to: C:\TWS API\source\pythonclient\TradingIdeas\Futures\06-reporting
- daily_account_ledger.csv, daily_account_ledger.parquet
- trade_impact_ledger.csv
- rolling_annualized_volatility.csv, rolling_annualized_volatility.parquet
- performance_summary.csv, monthly_returns.csv, yearly_returns.csv, yearly_drawdowns.csv
